# Skills requested in Google job posts

## Miguel Ángel Canela, IESE Business School

******


###  Introduction

Which **language, skills, and experience** should we add to our toolbox for getting a job in Google? Since Google publishes all of their jobs at `careers.google.com`, Niyamat Ullah scraped all of the job data from that site by going through every job page using a tool called **Selenium**, taking only the job title, location, responsibilities, minimum and preferred qualifications.

The data set, available in the Kaggle website, contains posts for 1,250 jobs. The variables are:

* `company`: either Google or Youtube.

* `title`: the title of the job.

* `category`: the category of the job.

* `location`: the location of the job.

* `responsibilities`: the responsibilities for the job.

* `minqual`: the minimum qualifications for the job.

* `prefqual`: the preferred qualifications for the job.

### Importing the data

I import the data from a remote CSV file. The encoding is typically specified for files that contain text, to prevent problems with systems that do not use UTF-8 (like Windows). This is probably not needed here, but it may save trouble in other cases.

In [1]:
import pandas as pd
fname = 'https://raw.githubusercontent.com/mcanela-iese/ML_Course/master/Data/' \
    'skills_google.csv'
df = pd.read_csv(fname, encoding='utf-8')

As usual, I check that the content of the file is what I expected.

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 7 columns):
company             1250 non-null object
title               1250 non-null object
category            1250 non-null object
location            1250 non-null object
responsibilities    1235 non-null object
minqual             1236 non-null object
prefqual            1236 non-null object
dtypes: object(7)
memory usage: 68.4+ KB


### Exploring the company

I start my exploratory analysis by counting the occurrences of the unique values of `company`. In most of the jobs, the company is Google.

In [3]:
df['company'].value_counts()

Google     1227
YouTube      23
Name: company, dtype: int64

### Exploring the job titles

There are 794 different job titles included in the data set.

In [4]:
title = df['title']
title.unique().shape

(794,)

I extract the top-10 titles.

In [5]:
title.value_counts()[:10]

Business Intern 2018                          35
MBA Intern, Summer 2018                       34
MBA Intern 2018                               28
BOLD Intern, Summer 2018                      21
Field Sales Representative, Google Cloud      17
Interaction Designer                          12
User Experience Researcher                     9
User Experience Design Intern, Summer 2018     7
Partner Sales Engineer, Google Cloud           7
Recruiter                                      7
Name: title, dtype: int64

The internships seem to dominate the picture, but, with 794 different titles, this quick view could be misleading. So, I check other possibilities. 

In [6]:
title.str.contains(pat='Intern').sum()

187

In [7]:
title.str.contains(pat='Sales').sum()

135

In [8]:
title.str.contains(pat='Cloud').sum()

277

In [9]:
title.str.contains(pat='Google Cloud').sum()

259

This analysis shows that, looking at the titles themselves, we lose the focus. It is more interesting to look at the occurrence of key words. So far, the cloud looks like the main one. 

To proceed more systematically, I am going to extract a list of most frequent tokens. Previous to the extraction, I clean the data deleting all the expressions within parentheses. To get that, I use the function `str.replace`, which has two arguments, the **pattern** to replace (`pat`) and the **replacement** (`repl`). 

For the pattern, we can use a **regular expression**. In this case, I set `pat=' [(].+[)]'`. Let me provide some technical detail to make it more palatable: 

1. In a regular expression, the parentheses are used for grouping pieces of text. To refer to the parentheses temselves, I use the square brackets.

2. The dot (.) stands for any character.

3. The plus symbol (+) is a **quantifier**, meaning any number of occurrences.

In [10]:
title = title.str.replace(pat=' [(].+[)]', repl='')

Next, I am going split the composite titles, which have two parts separated by a comma. I will use the function `str.split`, which takes as the argument the pattern that separates the pieces (`pat`). As shown by the following line of code, this function returns a series whose terms are lists of different lengths, which makes it difficult to manage. 

In [11]:
title.str.split(pat=', ').head()

0                       [Google Cloud Program Manager]
1     [Supplier Development Engineer, Cable/Connector]
2    [Data Analyst, Product and Tools Operations, G...
3            [Developer Advocate, Partner Engineering]
4          [Program Manager, Audio Visual Deployments]
Name: title, dtype: object

I need a workaround to get a series whose terms are single strings. First, I transform the series returned by `str.split` into a list. 

In [12]:
L = list(title.str.split(pat=', '))
L[:5]

[['Google Cloud Program Manager'],
 ['Supplier Development Engineer', 'Cable/Connector'],
 ['Data Analyst', 'Product and Tools Operations', 'Google Technical Services'],
 ['Developer Advocate', 'Partner Engineering'],
 ['Program Manager', 'Audio Visual Deployments']]

Now, I flatten the list `L` so that its terms are singkle strings.

In [13]:
title = L[0]
for x in L[1:]:
    title = title + x
title[:5]

['Google Cloud Program Manager',
 'Supplier Development Engineer',
 'Cable/Connector',
 'Data Analyst',
 'Product and Tools Operations']

Now, `title` is a list of 2,353 job titles.

In [14]:
len(title)

2353

Finally, I transform `title` into a Pandas series, to be able to use the function `value_counts`. 

In [15]:
pd.Series(title).value_counts()[:10]

Google Cloud                  206
Summer 2018                    76
Business Intern 2018           51
MBA Intern                     34
MBA Intern 2018                32
Google Technical Services      31
Consumer Hardware              27
Field Sales Representative     26
Google Cloud Platform          25
Product Marketing Manager      23
dtype: int64

What do we get, at the end of the day? That most of the jobs are for Google Cloud or for interns. The sales jobs seem to be very scattered, with many different title definitions.

### Exploring categories

I follow the same approach for categories. The conclusion, here, is that technical jobs are a minority.

In [16]:
df['category'].value_counts()[:15]

Sales & Account Management      168
Marketing & Communications      165
Finance                         115
Technical Solutions             101
Business Strategy                98
People Operations                86
User Experience & Design         84
Program Management               74
Partnerships                     60
Product & Customer Support       50
Legal & Government Relations     46
Administrative                   40
Sales Operations                 31
Software Engineering             31
Hardware Engineering             26
Name: category, dtype: int64

### Exploring countries

To learn in which countries the posts are located, I extract the country from the location. The location has two or three components, separated by the string formed by a comma followed by a white space (', '). 

In [17]:
country = df['location']
country.head()

0                           Singapore
1                     Shanghai, China
2         New York, NY, United States
3    Mountain View, CA, United States
4        Sunnyvale, CA, United States
Name: location, dtype: object

So I have to drop all the characters that come before the last occurrence of that string. I use the function `str.replace` and, as pattern, a regular expression which stands for all the strings to be suppressed, that is, for any string ending by a comma followed by white space.

In [18]:
country = country.str.replace(pat='.+, ', repl='')

There are 49 unique values but, as we see next, most of the job requests are for United States.

In [19]:
country.unique().shape

(49,)

In [20]:
country.value_counts()[:10]

United States     638
Ireland            87
United Kingdom     62
Germany            54
Singapore          41
China              38
Australia          35
Japan              31
Taiwan             30
India              28
Name: location, dtype: int64

### Exploring responsibilities

Now I explore the content of the responsibilities column, which has **missing values**. To check this, I apply the function `isna`. Note that this function returns a Boolean series, which is coerced to a numeric series when we we perform any calculation.

In [21]:
resp = df['responsibilities']
resp.isna().sum()

15

In [22]:
resp = resp.dropna()

Next, I put everything in **lowercase** with the function `str.lower`.

In [23]:
resp = resp.str.lower()

Let us see how the text looks like.

In [24]:
resp.iloc[0]

'shape, shepherd, ship, and show technical programs designed to support the work of cloud customer engineers and solutions architects.\nmeasure and report on key metrics tied to those programs to identify any need to change course, cancel, or scale the programs from a regional to global platform.\ncommunicate status and identify any obstacles and paths for resolution to stakeholders, including those in senior roles, in a transparent, regular, professional and timely manner.\nestablish expectations and rationale on deliverables for stakeholders and program contributors.\nprovide program performance feedback to teams in product, engineering, sales, and marketing (among others) to enable efficient cross-team operations.'

Next, I extract the words from the text. This will leave out **punctuation** and the **control character** '\n', which means new line, and is used to separate paragraphs. I use a regular expression that stands for any word with the function `str.findall`, which returns a data frame whose elements are lists of different lengths. Each of these lists is a **bag of words**. To collect all the words together in a single list, I use a workaround, similar to that used for the job titles. 

In [25]:
L = resp.str.findall(pat='[a-z]+')
L[:5]

0    [shape, shepherd, ship, and, show, technical, ...
1    [drive, cross, functional, activities, in, the...
2    [collect, and, analyze, data, to, draw, insigh...
3    [work, one, on, one, with, the, top, android, ...
4    [plan, requirements, with, internal, customers...
Name: responsibilities, dtype: object

In [26]:
resp = L[0]
for x in L[1:]:
    resp = resp + x
resp[:5]

['shape', 'shepherd', 'ship', 'and', 'show']

Now, `resp` is a list of 109,545 terms.

In [27]:
len(resp)

109745

By transforming `resp` into a Pandas series, I can use `value_counts`. 

In [28]:
pd.Series(resp).value_counts()[:25]

and          9457
to           4303
the          2668
of           2233
with         2182
for          1372
google       1292
in           1247
business     1218
a            1185
product       968
on            870
develop       779
teams         768
work          755
as            712
team          660
partners      633
technical     606
manage        596
customer      561
our           536
that          517
partner       516
support       489
dtype: int64

Most of these terms are **stopwords**, that is, words that do not contain relevant information (and, to, the, etc). Others, like 'Google', do not provide information in the context of this analysis. The leading topics seem to be development, teams and partners. To get a better picture, I should continue the analysis by dropping the stopwords and merging **synonyms** (such as "team" and "teams"). But I stop here, to make it short.

### Exploring minimum qualifications

The analysis of the minimum qualifications follows the same lines.

In [29]:
minqual = df['minqual']
minqual.isna().sum()

14

In [30]:
minqual = minqual.dropna()
minqual = minqual.str.lower()
minqual.iloc[0]

'ba/bs degree or equivalent practical experience.\n3 years of experience in program and/or project management in cloud computing, enterprise software and/or marketing technologies.'

In [31]:
L = minqual.str.findall(pat='[a-z]+')
L[:5]

0    [ba, bs, degree, or, equivalent, practical, ex...
1    [bs, degree, in, an, engineering, discipline, ...
2    [bachelor, s, degree, in, business, economics,...
3    [ba, bs, degree, in, computer, science, or, eq...
4    [ba, bs, degree, or, equivalent, practical, ex...
Name: minqual, dtype: object

In [32]:
minqual = L[0]
for x in L[1:]:
    minqual = minqual + x
minqual[:5]

['ba', 'bs', 'degree', 'or', 'equivalent']

In [33]:
len(minqual)

46929

In [34]:
pd.Series(minqual).value_counts()[:25]

experience    3036
or            2478
in            2400
and           2304
a             1231
of            1110
equivalent    1063
degree        1059
practical      993
to             928
bs             879
ba             838
years          722
with           718
the            611
management     413
ability        363
field          341
related        321
working        313
program        305
as             292
english        286
speak          281
fluently       280
dtype: int64

Bad news here for the starters, experience is the main thing. I check how often are mentioned the leading programming languages. Not much.

In [35]:
pd.Series(minqual).str.contains(pat='sql').sum()

85

In [36]:
pd.Series(minqual).str.contains(pat='javascript').sum()

77

In [37]:
pd.Series(minqual).str.contains(pat='python').sum()

97

### Exploring preferred qualifications

I replicate the analysis for the preferred qualifications.

In [38]:
prefqual = df['prefqual']
prefqual.isna().sum()

14

In [39]:
prefqual = prefqual.dropna()
prefqual = prefqual.str.lower()
prefqual.iloc[0]

'experience in the business technology market as a program manager in saas, cloud computing, and/or emerging technologies.\nsignificant cross-functional experience across engineering, sales, and marketing teams in cloud computing or related technical fields.\nproven successful program outcomes from idea to launch in multiple contexts throughout your career.\nability to manage the expectations, demands and priorities of multiple internal stakeholders based on overarching vision and success for global team health.\nability to work under pressure and possess flexibility with changing needs and direction in a rapidly-growing organization.\nstrong organization and communication skills.'

In [40]:
L = prefqual.str.findall(pat='[a-z]+')
L[:5]

0    [experience, in, the, business, technology, ma...
1    [bsee, bsme, or, bsie, degree, experience, of,...
2    [experience, partnering, or, consulting, cross...
3    [experience, as, a, software, developer, archi...
4    [cts, certification, experience, in, the, cons...
Name: prefqual, dtype: object

In [41]:
prefqual = L[0]
for x in L[1:]:
    prefqual = prefqual + x
prefqual[:5]

['experience', 'in', 'the', 'business', 'technology']

In [42]:
len(prefqual)

83850

In [43]:
pd.Series(prefqual).value_counts()[:25]

and              6496
to               2991
in               2501
with             2419
experience       2308
ability          1856
of               1655
a                1654
skills           1461
or               1432
the              1379
management        720
business          680
demonstrated      640
as                582
excellent         570
work              549
communication     546
environment       520
technical         476
strong            472
analytical        446
working           440
cloud             395
data              394
dtype: int64

In [44]:
pd.Series(prefqual).str.contains(pat='sql').sum()

84

In [45]:
pd.Series(prefqual).str.contains(pat='javascript').sum()

66

In [46]:
pd.Series(prefqual).str.contains(pat='python').sum()

79